In [1]:
import sys, psycopg2
import pandas as pd
sys.path.append('C:\\Projects\\uni\\sources\\companybot\\bots')
from datetime import datetime
from config import DB_NAME, DB_HOST, DB_USER, DB_PASSWORD, CRUNCHBASE_DIR
from common import get_data_diff, write_organizations_pending, get_uuids_from_crunchbase_organizations, DataSource, PendingStatus
from companies_house_bot import CompaniesHouseBot, run_companies_house_bot
from crunchbase_bot import run_crunchbase_bot
from linkedin_bot import run_linkedin_bot

## write all data based on list of names

### get uuids by name

In [6]:
names = ['elliptic', 'isize']
df_uuids = pd.DataFrame(get_uuids_from_crunchbase_organizations(names))
uuids = list(df_uuids['uuid'])
df_uuids[['name', 'founded_on']]

,name,founded_on
0,Elliptic,2013-10-01
1,Elliptical,None
2,iSIZE Technologies,2016-09-22


In [7]:
pd.DataFrame(CompaniesHouseBot.get_data_from_pending(uuids=uuids, force=True))

,uuid,uuid_parent,name,legal_name,category_groups_list,founded_on,source,status,version,created_at,updated_at,crunchbase_data
0,5fd4dcb9-77ec-8903-5300-dd4c3da76670,5fd4dcb9-77ec-8903-5300-dd4c3da76670,Elliptic,Elliptic Enterprises Ltd.,"[Financial Services, Other, Payments, Software]",2013-10-01,companies_house,pending,,2023-05-25 12:44:07,2023-05-25 12:44:07,"{'properties': {'company_type': 'for_profit', ..."
1,a7d2f427-66ba-476a-81cc-171a5d806b22,a7d2f427-66ba-476a-81cc-171a5d806b22,iSIZE Technologies,iSIZE Limited.,"[Artificial Intelligence, Content and Publishi...",2016-09-22,companies_house,pending,,2023-05-25 12:15:53,2023-05-25 12:15:53,"{'properties': {'company_type': 'for_profit', ..."


### write uuids to pending table

In [8]:
### uuids needs to be existing in pending data for any bot to pick them up. bots can only see the data in pending table
write_organizations_pending(uuids=uuids,
                            category_groups_list='*',
                            source=DataSource.crunchbase,
                            status=PendingStatus.pending,
                            from_dt=datetime.strptime('2012-01-01', '%Y-%m-%d'),
                            to_dt=datetime.strptime('2018-01-01', '%Y-%m-%d'),
                            force=False)

write_organizations_pending successful: source: crunchbase status: PendingStatus.pending rows: 2


### run crunchbase bot

In [9]:
# crunchbase bot will run in force mode and overwrite data table as well as write for companies_house into pending
run_crunchbase_bot(uuids=uuids, force=True)

write data successful: source: crunchbase status: completed company: Elliptic
write data successful: source: crunchbase status: completed company: iSIZE Technologies


### run companies house bot

In [10]:
# companies house bot will run in force mode and overwrite data table as well as weite for linkedin data into pending
run_companies_house_bot(uuids=uuids, force=True)


searching for {"company_name_CB": "ELLIPTIC ENTERPRISES LTD.", "founded_on_CB": "2013-10-01", "founders_CB": ["Adam Joyce", "James Smith", "Tom Robinson"]}
successful match by FOUNDER NAME and DATE {"company_name_CB": "ELLIPTIC ENTERPRISES LTD.", "company_name_CH": "ELLIPTIC ENTERPRISES LIMITED", "founded_on_CB": "2013-10-01", "founded_on_CH": "2013-03-22", "founders_CB": ["Adam Joyce", "James Smith", "Tom Robinson"], "officers_CH": ["Edward Lascelles", "Simone Maini", "Christopher Pilling", "Richard Seewald", "James Smith", "Zihao Xu", "Rebecca Hunt", "Adam Joyce", "Walter Kortschak", "Tomoyuki Nii", "Eyal Rabinovich", "Thomas Robinson", "Christopher Scoggins", "Christopher Steed"], "url_CH": "https://find-and-update.company-information.service.gov.uk/company/08458210"}
FOUND and starting to crawl{"company_name_CB": "ELLIPTIC ENTERPRISES LTD.", "founded_on_CB": "2013-10-01", "founders_CB": ["Adam Joyce", "James Smith", "Tom Robinson"], "url": "https://find-and-update.company-informat

C:\Users\Djordje\miniconda3\lib\site-packages\scrapy\utils\request.py:232: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting for information on how to handle this deprecation.
  return cls(crawler)


parse_insolvency company:Elliptic url: https://find-and-update.company-information.service.gov.uk/company/08458210/insolvency
parse_insolvency completed 1 {"url": "https://find-and-update.company-information.service.gov.uk/company/08458210/insolvency", "items": []}
parse_company_info company:Elliptic url: https://find-and-update.company-information.service.gov.uk/company/08458210
parse_company_info completed 2 {"company_name": "ELLIPTIC ENTERPRISES LIMITED", "company_id": "08458210", "url": "https://find-and-update.company-information.service.gov.uk/company/08458210", "key": "company", "address": "Office 7 35-37 Ludgate Hill, London, England, EC4M 7JN", "status": "Active", "type": "Private limited Company", "incorporated_on": "2013-03-22", "sics": ["62090 - Other information technology service activities"]}


C:\Users\Djordje\miniconda3\lib\site-packages\scrapy\selector\unified.py:83: UserWarning: Selector got both text and root, root is being ignored.
  super().__init__(text=text, type=st, root=root, **kwargs)


parse_filing company: Elliptic url: https://find-and-update.company-information.service.gov.uk/company/08458210/filing-history
parse_confirmation_statement: 2023-03-22 https://find-and-update.company-information.service.gov.uk/company/08458210/filing-history/MzM3NzQ2NTc2NWFkaXF6a2N4/document?format=pdf&download=0
parse_confirmation_statement completed: 2023-03-22 https://find-and-update.company-information.service.gov.uk/company/08458210/filing-history/MzM3NzQ2NTc2NWFkaXF6a2N4/document?format=pdf&download=0
parse_confirmation_statement: 2022-03-22 https://find-and-update.company-information.service.gov.uk/company/08458210/filing-history/MzMzNzM2NzI5NmFkaXF6a2N4/document?format=pdf&download=0
parse_confirmation_statement completed: 2022-03-22 https://find-and-update.company-information.service.gov.uk/company/08458210/filing-history/MzMzNzM2NzI5NmFkaXF6a2N4/document?format=pdf&download=0
parse_officers company: Elliptic url: https://find-and-update.company-information.service.gov.uk/comp

...

C:\Users\Djordje\miniconda3\lib\site-packages\scrapy\selector\unified.py:83: UserWarning: Selector got both text and root, root is being ignored.
  super().__init__(text=text, type=st, root=root, **kwargs)


parse_confirmation_statement completed: 2018-09-21 https://find-and-update.company-information.service.gov.uk/company/10388803/filing-history/MzIxNzA5NjMxNGFkaXF6a2N4/document?format=pdf&download=0
parse_confirmation_statement: 2017-09-21 https://find-and-update.company-information.service.gov.uk/company/10388803/filing-history/MzE4NjYwMjc1NGFkaXF6a2N4/document?format=pdf&download=0
parse_confirmation_statement completed: 2017-09-21 https://find-and-update.company-information.service.gov.uk/company/10388803/filing-history/MzE4NjYwMjc1NGFkaXF6a2N4/document?format=pdf&download=0
parse_filing company: iSIZE Technologies url: https://find-and-update.company-information.service.gov.uk/company/10388803/filing-history?page=4
parse_incorporation: 2016-09-22 https://find-and-update.company-information.service.gov.uk/company/10388803/filing-history/MzE1Nzc4NTQ1N2FkaXF6a2N4/document?format=pdf&download=0


C:\Users\Djordje\miniconda3\lib\site-packages\scrapy\selector\unified.py:83: UserWarning: Selector got both text and root, root is being ignored.
  super().__init__(text=text, type=st, root=root, **kwargs)


parse_filing company: iSIZE Technologies url: https://find-and-update.company-information.service.gov.uk/company/10388803/filing-history?page=5
parse_filing completed 5 {"url": "https://find-and-update.company-information.service.gov.uk/company/10388803/filing-history", "items": []}
parse_finished
get_persons {"company_id": "10388803", "company_name": "ISIZE LIMITED", "items": [{"name": "Charles Conn Iii", "full_name": "Conn Iii, Charles Ronald", "occupation": ["Director", "Shareholder"], "date_of_birth": "1961-08-01"}, {"name": "Edward Clifton", "full_name": "Clifton, Edward", "occupation": ["Director"], "date_of_birth": "1971-01-01"}, {"name": "Hugo Muriel Arriaran", "full_name": "Muriel Arriaran, Hugo Marcelo", "occupation": ["Director", "Shareholder"], "date_of_birth": "1983-10-01"}, {"name": "Ioannis Andreopoulos", "full_name": "Andreopoulos, Ioannis", "occupation": ["Director", "Shareholder"], "date_of_birth": "1977-01-01"}, {"name": "Marko Milos", "full_name": "Milos, Marko", "o

C:\Users\Djordje\miniconda3\lib\site-packages\scrapy\selector\unified.py:83: UserWarning: Selector got both text and root, root is being ignored.
  super().__init__(text=text, type=st, root=root, **kwargs)


### run linkedin bot

In [ ]:
run_linkedin_bot(uuids=uuids, force=True)

### get differences between crunchbase and companies_house data

In [3]:
pd.DataFrame(get_data_diff())

,uuid,name,source,category_groups_list
0,0873546f-63ae-448e-aa1a-8506883736e9,AdamEve&Apple,crunchbase,"[Advertising, Artificial Intelligence, Data an..."
1,0f26fcc3-dd5d-439e-9f30-6c6a7301379a,Agile Boards,crunchbase,"[Artificial Intelligence, Data and Analytics, ..."
2,1c5b6e05-e3ec-4208-9cf6-e708ce554c3f,Agile Automations,crunchbase,"[Artificial Intelligence, Data and Analytics, ..."
3,254cf148-6b4a-4186-8430-4d497e069d66,Affine,crunchbase,"[Artificial Intelligence, Data and Analytics, ..."
4,2a8b92f7-39e2-4b12-8f27-85d0b7a45cb9,Adarga,crunchbase,"[Artificial Intelligence, Data and Analytics, ..."
5,300b75bc-2d9b-4adb-be73-e7a1967a35f7,Acronotics,crunchbase,"[Artificial Intelligence, Data and Analytics, ..."
6,3313c152-4796-5b38-db11-3ed56c1cf592,Adthena,crunchbase,"[Artificial Intelligence, Data and Analytics, ..."
7,5a67df18-bc90-417c-a605-d309a902f9af,Adverttu,crunchbase,"[Advertising, Artificial Intelligence, Data an..."
8,658dad77-fb5b-406e-8326-5fb0016eacf0,AI-Health,crunchbase,"[Apps, Artificial Intelligence, Data and Analy..."
9,6722fb72-f597-45c1-9fbb-c724efda577f,AI Community Job Board,crunchbase,"[Administrative Services, Artificial Intellige..."


In [20]:
def is_company(name, company_identifiers=['LIMITED', 'LTD', 'L.T.D.', 'PLC', 'P.L.C.', 'L.P.', 'GROUP', 'INVESTMENT',
                                          'INVESTMENTS', 'CAPITAL', 'CORP', 'CORPORATION', 'COMPANY', 'PARTNER', 'EQUITY',
                                          'VENTURE', 'VENTURES', 'LP', 'LLP', 'L.P.', 'L.L.P.',
                                          'LLC', 'L.L.C.', 'STARTUP', 'GMBH', 'G.M.B.H', 'SARL', 'S.A.R.L', 'S.A.R.L.', 'FONDATION',
                                          'FOUNDATION', '&', 'TRUST', 'UNIVERSITY', 'SCHOOL', 'SUPPORT', 'SAS', 'S.A.S.',
                                          'NOMINEE', 'TRADING', 'AS', 'MANAGEMENT']):
    for id in company_identifiers:
        pattern = r"\b" + re.escape(id.lower()) + r"\b"
        if re.search(pattern, name.lower()):
            return True

    return False

In [21]:
import re
is_company('ASDAS MANAGEMENT')

True